In [1]:
from gensim import models

In [48]:
import pandas as pd
import re

In [3]:
class Tenders_W2V:
    def __init__(self):
        self.model = models.KeyedVectors.load_word2vec_format(
            '../W2V/GoogleNews-vectors-negative300-SLIM.bin', binary=True)
    def get_relation(self, x, y):
        if x == y: return 1
        value = 0
        try:
            value = self.model.rank(x, y)
        except:
            pass
        return value

In [6]:
twv = Tenders_W2V()

In [4]:
twv.get_relation('happy', 'sad')

25

In [4]:
tenders_df = pd.read_csv('Tenders/tmp_tenders.csv')[['_id','key_0','key_orig_0','key_1','key_orig_1','key_2','key_orig_2','key_3','key_orig_3','key_4','key_orig_4']]

In [2]:
tenders_df = pd.read_csv('remove_num_tenders.csv')

In [58]:
re_col = ['_id', 'key_0', 'key_1', 'key_2', 'key_3', 'key_4']
re_orig_col = ['_id', 'key_orig_0', 'key_orig_1', 'key_orig_2', 'key_orig_3', 'key_orig_4']
# for i in tenders_df.columns:
#     if 'orig' in i: re.append(i)
input_df = tenders_df[re_col]

In [55]:
input_df.iloc[0].values.tolist()

['6162aa1fe1b7f5c73e6fdfd0',
 'inform',
 'infrastructur',
 'research',
 'nation',
 'noie']

In [4]:
word_df = input_df.melt(id_vars = '_id')

In [5]:
input_df[input_df['_id']=='6162aa1fe1b7f5c73e6fe38a']

,_id,key_0,key_1,key_2,key_3,key_4
954,6162aa1fe1b7f5c73e6fe38a,labour,model,suppli,gener,decis


In [66]:
map_df = word_df.groupby('value')['_id'].apply(lambda x: list(set(x))).reset_index()

In [35]:
map_df.to_csv('map_df.csv', index = 0)

In [36]:
# map_df = pd.read_csv('map_df.csv')

In [67]:
map_df

,value,_id
0,aaa,[6162aa1fe1b7f5c73e6fe1a8]
1,aad,[6162aa1fe1b7f5c73e6fe51b]
2,aai,[6162aa1fe1b7f5c73e6fea46]
3,aass,[6162aa1fe1b7f5c73e6fe537]
4,aaswa,[6162aa1fe1b7f5c73e6fe9c4]
...,...,...
2497,yield,"[6162aa1fe1b7f5c73e6fe82f, 6162aa1fe1b7f5c73e6..."
2498,young,"[6162aa1fe1b7f5c73e6fe952, 6162aa1fe1b7f5c73e6..."
2499,youth,"[6162aa1fe1b7f5c73e6fe0c8, 6162aa1fe1b7f5c73e6..."
2500,zealand,[6162aa1fe1b7f5c73e6fe4f5]


In [73]:
test_df = tenders_df.sample(1)
test_df

,index,ATM ID,Category,Description,Title,_id,raw_result,key_0,key_1,key_2,key_3,text,key_4
2636,2636,FIN/007/20Notice7,Management advisory services,This notice is to provide interested parties w...,Industry Update – Management Advisory Services...,6162aa1fe1b7f5c73e6fea1c,"[('phase commercial australian', 0.7604), ('co...",servic,advisori,manag,panel,this notice is to provide interested parties w...,australian


In [74]:
result_list = []
key_df = input_df[input_df['_id'] == test_df['_id'].values[0]]
reg = r'\'(.*?)\''
for i in list(key_df.columns)[1:]:
#     result = re.findall(reg, map_df[map_df['value'] == key_df[i].values[0]]['_id'].values[0])
    result = map_df[map_df['value'] == key_df[i].values[0]]['_id'].values[0]
    result_list.append(result)

In [75]:
key_0_set = set(result_list[0])
key_0_set.remove(test_df['_id'].values[0])
cnt_dict = {0:key_0_set}
for idx in range(1, len(result_list)):
    cnt_dict[idx] = cnt_dict[idx-1].intersection(set(result_list[idx]))

In [86]:
cnt_dict[2]

{'6162aa1fe1b7f5c73e6fe9cf',
 '6162aa1fe1b7f5c73e6fe9d8',
 '6162aa1fe1b7f5c73e6fe9d9',
 '6162aa1fe1b7f5c73e6fea2a',
 '6162aa1fe1b7f5c73e6fea54',
 '6162aa1fe1b7f5c73e6fea55',
 '6162aa1fe1b7f5c73e6fea56'}

In [91]:
tenders_df[tenders_df['_id'] == '6162aa1fe1b7f5c73e6fea55']['Description'].unique()

array(['This notice is to inform interested parties about a proposed whole-of-Australian Government Management Advisory Services Panel.'],
      dtype=object)

In [88]:
test_df['Description'].unique()

array(['This notice is to provide interested parties with information about Phase 2 – Corporate and Phase 3 - Commercial of the Management Advisory Services Panel for the Australian Government.'],
      dtype=object)

In [12]:
word_df['index'] = 0

In [13]:
outer_df = word_df.merge(word_df, on ='index',how='outer')

In [14]:
outer_df = outer_df[outer_df['value_x'] != outer_df['value_y']]

In [15]:
def get_distance(row):
    return twv.get_relation(row['value_x'], row['value_y'])

In [18]:
outer_df['value'] = outer_df.apply(lambda x: get_distance(x), axis=1)

In [21]:
outer_df.sort_values('value', ascending=False)

,index,value_x,value_y,value
1523,0,perth,project,298737
498,0,project,perth,298518
222,0,australia,fund,298341
509,0,fund,australia,298126
711,0,au,guidelines,295542
...,...,...,...,...
495,0,project,nii,0
1201,0,productivity,raaf,0
487,0,project,raaf,0
1209,0,productivity,nii,0
